In [1]:
from contextlib import contextmanager
import os
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import win32com.client as wc
from typing import List, Optional

# Load environment variables
def load_env_vars() -> object:
    try:
        current_directory = (
            Path(__file__).resolve().parent if "__file__" in locals() else Path.cwd()
        )
        environment_variables = current_directory / ".env"
        load_dotenv(environment_variables)
        USER = os.getenv("USER")
        PASSWORD = os.getenv("PASSWORD")
        HOST = os.getenv("HOST")
        PORT = os.getenv("PORT")
        DATABASE = os.getenv("SERVICE_NAME")
        connection_string = f"oracle+oracledb://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
        print("Environmental variables loaded.")
        return connection_string
    except Exception as error:
            print(f"An error occurred: {error}")
            return None

# Create database connection
@contextmanager
def create_db_connection(connection_string: str):
    engine = create_engine(connection_string)
    connection = engine.connect()
    try:
        yield connection
    except SQLAlchemyError as error:
        print(f"An error occurred: {error}")
        return None

# Read SQL script
def read_sql_script(sql_path: str) -> object:
    try:
        with open(sql_path) as file:
            query = file.read()
            print("SQL query read.")
            return query
    except Exception as error:
        print(f"An error occurred: {error}")
        return None

# Execute query
def execute_query(connection: str, query: str) -> object:
    try:
        data = pd.read_sql(text(query), connection)
        print("Data returned.")
        return data
    except Exception as error:
        print(f"An error occurred: {error}")
        return None

# Output data to spreadsheet
def write_to_excel(data: pd.DataFrame, output_filepath: str) -> None:
    with pd.ExcelWriter(
        output_filepath,
        engine="xlsxwriter",
        datetime_format="dd mmm yyyy hh:mm:ss",
        date_format="dd mmm yyyy",
    ) as writer:
        data.to_excel(writer, sheet_name="Sheet1", index=False, header=False, startrow=1,)
        workbook = writer.book
        worksheet = writer.sheets["Sheet1"]
        header_format = workbook.add_format(
            {
                "bold": True,
                "text_wrap": True,
                "valign": "top",
                "fg_color": "#9FC5E8",
                "border": 1,
            }
        )
        for col_num, value in enumerate(data.columns):
            worksheet.write(0, col_num, value, header_format)
        print("Spreadsheet created.")

# Email function
def send_email(mail_from: str, mail_to: List[str], subject: str, mail_body: str, mail_attachments: List[str], mail_cc: Optional[List[str]] = None) -> None:
    try:
        outlook = wc.Dispatch("Outlook.Application")
        mail = outlook.CreateItem(0)
        mail.SentOnBehalfOfName = mail_from
        mail.To = "; ".join(mail_to)
        if mail_cc is not None:
            mail.CC = "; ".join(mail_cc)
        mail.Subject = subject
        mail.HTMLBody = mail_body
        for attachment in mail_attachments:
            mail.Attachments.Add(attachment)
        mail.Display()
        print("Email created.")
    except Exception as error:
        print(f"An error occurred: {error}")
    
# Main program
def main():
    connection_string = load_env_vars()
    with create_db_connection(connection_string) as connection:
        sql_path = Path("./sql/query.sql")
        query = read_sql_script(str(sql_path))
        if query:
            data = execute_query(connection, query)
            output_filepath = Path("./output/test.xlsx")
            # set attachment
            write_to_excel(data, str(output_filepath))
            send_email()
if __name__ == "__main__":
    main()

AttributeError: module 'pandas' has no attribute 'Dataframe'